In [ ]:
# https://www.tensorflow.org/lite/tutorials/model_maker_image_classification
from __future__ import absolute_import, division
from BashColors import C
from CV2_Utils_2 import *
from TarfileFunctions import *

from time import perf_counter, sleep
import itertools, json, os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'

import numpy as np
import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

# from tflite_support.metadata_writers import image_classifier
# from tflite_support.metadata_writers import writer_utils

import matplotlib.pyplot as plt

contentPath=os.getcwd()
image_path=os.path.join(contentPath, 'images')
generatorPath=os.path.join(contentPath, 'DataGenerator')
imagePath=join(contentPath, '3b7d7d8a64.jpg')
modelPath=join(contentPath, 'Defcon4_mobilenet_v2_FLOAT16.tflite')

initialGlobList:list
with open("initialGlobList.json", 'r') as f:
    initialGlobList = json.load(f)
    
def listNewFiles(initial=initialGlobList, delete=False):
    currentFilesGlob=glob.glob('**')
    print('New files...')
    if len(initial) == len(currentFilesGlob):
        print(f'{C.BIRed}Nothing to see here.\nNo new files.')
    for fil in currentFilesGlob:
        if not fil in initial:
            if isdir(fil):
                print(f'{C.BIBlue}{fil}')
                if delete:
                    shutil.rmtree(fil)
            elif isfile(fil):
                print(f'{C.ColorOff}{fil}')
                if delete:
                    os.remove(fil)
listNewFiles()

In [ ]:
class TestModel(tf.Module):
    def __init__(self):
        super(TestModel, self).__init__()

    @tf.function(input_signature=[tf.TensorSpec(shape=[1, 10],
                                                dtype=tf.float32)])
    def add(self, x):
        '''
        Simple method that accepts single input 'x' and returns 'x' + 4.
        '''
        # Name the output 'result' for convenience.
        return {'result' : x + 4}

SAVED_MODEL_PATH = contentPath
TFLITE_FILE_PATH = modelPath

# Save the model
module = TestModel()
# You can omit the signatures argument and a default signature name will be
# created with name 'serving_default'.
tf.saved_model.save(
    module, SAVED_MODEL_PATH,
    signatures={'my_signature':module.add.get_concrete_function()})

# Convert the model using TFLiteConverter
converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL_PATH)
tflite_model = converter.convert()
with open(TFLITE_FILE_PATH, 'wb') as f:
  f.write(tflite_model)

# Load the TFLite model in TFLite Interpreter
interpreter = tf.lite.Interpreter(TFLITE_FILE_PATH)
# There is only 1 signature defined in the model,
# so it will return it by default.
# If there are multiple signatures then we can pass the name.
my_signature = interpreter.get_signature_runner()

# my_signature is callable with input as arguments.
output = my_signature(x=tf.constant([1.0], shape=(1,10), dtype=tf.float32))
# 'output' is dictionary with all outputs from the inference.
# In this case we have single output 'result'.
print(output['result'])

In [ ]:
 def testTFLiteModel(thisModelPath, inputImage):
    '''thisModelPath: fully qualified path to tflite model.'''
    interpreter = tf.lite.Interpreter(model_path=thisModelPath)
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.allocate_tensors()
    
    # new_img = np.copy(inputImage)
    new_img = cv2.imread(imagePath, cv2.IMREAD_COLOR)
    input_data = np.expand_dims(new_img, axis=0).astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)

    interpreter.invoke()
    return interpreter
# help(testTFLiteModel)
print('ok')

print(modelPath)
print(imagePath)
interpreter=testTFLiteModel(modelPath, image_path)

In [ ]:
interpreter.get_output_details()